# Points of interest (3 pt.)

Закомментированная часть - пример добавления своего фильтра. Фильтр Харриса возвращает очень маленькие значения, нужно придумать, как их визуализировать и подобрать параметры фильтра, чтобы он находил небольшое количество наиболее значимых точек интереса.

Для ускорения вычислений размер картинки уменьшается в 3 раза по каждому измерению, с этим можно экспериментировать.

In [ ]:
import cv2
import time
import os
import math
import sys
from matplotlib import pyplot as plt

sys.path.append("/Users/elijah/Dropbox/Programming/adelaida/modules/")

import input_output
import processor

det = processor.Processors ()
det.add_processor ("keypoints extr")

det.add_filter (processor.resize (downscale_factor_ = 3),
                "keypoints extr", "downscale")
det.add_filter (processor.colorspace_to_colorspace ("RGB", "GRAY"),
                "keypoints extr", "colorspaces change")


# bsz = 1 #block size
# asz = 3 #aperture size
# k   = 0.01 #alg parameter

# det.add_filter (processor.custom_operation (lambda img : cv2.cornerHarris
#     (img, bsz, asz, k), "harris filter"), "keypoints extr", "harris filter")

source  = input_output.Source ("-1")

while (True):
    frame = source.get_frame ()

    result, success = det.process (frame, "keypoints extr")

    cv2.imshow ("harris", result)
    
    time.sleep (0.02)

    keyb = cv2.waitKey (1) & 0xFF

    if (keyb == ord('q')):
        break

cv2.destroyAllWindows()

# Filtering&Hand extraction (4 pt.)

Copy the example from the seminar, add properly used filter_connected_components filter and morphology filter from processors.py and tune the ranges so that you can robustly detect object of your choice. It can be you hand, your favorite mug, colored sticker, anything. Try different approaches :)